In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 Para saber a data da próxima Imersão IA com Google Gemini da Alura, a melhor forma é verificar diretamente no site da Alura. Você pode encontrar a informação nos seguintes locais:

*   **Página de Imersões:** Procure pela seção de Imersões ou Bootcamps no site da Alura. Lá devem estar listadas as próximas edições, incluindo a de IA com Google Gemini, se houver uma programada.
*   **Página do curso/formação específica:** Se já existe uma página dedicada à Imersão IA com Google Gemini, verifique lá as datas das próximas turmas.
*   **Canais de comunicação da Alura:** A Alura costuma divulgar as datas de novas edições em suas redes sociais, newsletters e outros canais de comunicação. Fique de olho!

In [5]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 Com base nos resultados da pesquisa, a Imersão IA com Google Gemini mais recente ocorreu entre os dias 12 e 16 de maio de 2025. As inscrições para essa edição já estão encerradas.

No momento, não há informações sobre uma próxima edição da Imersão IA com Google Gemini da Alura.

In [6]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini da Alura']
Páginas utilizadas na resposta: youtube.com



In [7]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [36]:
##########################################
# --- Agente 1: Mapeador de Recursos --- #
##########################################
def agente_mapeamento_recursos():
    mapeador = Agent(
        name="agente_mapeamento_recursos",
        model="gemini-2.0-flash", # Ou um modelo mais robusto se precisar de mais raciocínio
        instruction="""
        Você é um Agente de Mapeamento e Recursos. Sua tarefa é coletar, verificar e manter atualizadas informações
        sobre abrigos, locais de alimentação (refeitórios, sopas), postos de saúde, centros de assistência social (CRAS/CREAS),
        locais de banho público, lavanderias comunitárias e oportunidades de emprego para pessoas em situação de rua.
        Utilize a ferramenta Google Search para encontrar informações em sites de prefeituras, ONGs e outras organizações.
        Seja meticuloso para garantir a precisão dos dados (endereço, horário, requisitos de acesso, contato, vagas se possível).
        Retorne as informações de forma estruturada, listando o nome do recurso, tipo, endereço, horário e contato.
        """,
        description="Agente que mapeia e atualiza recursos para pessoas em situação de rua.",
        tools=[google_search] # Pode ser necessário adicionar ferramentas para interagir com um DB no futuro
    )
    # A entrada para este agente pode ser 'Buscar abrigos em [localidade]' ou 'Atualizar dados de [tipo de recurso]'
    # A saída deve ser um formato estruturado (lista de dicionários, JSON)
    # return call_agent(mapeador, "Busque os 5 principais abrigos em São Paulo com vagas.")
    return mapeador # Retorna o objeto do agente para ser usado em outras chamadas


In [37]:
################################################
# --- Agente 2: Interação de Necessidades --- #
################################################
def agente_interacao_necessidades():
    interacao = Agent(
        name="agente_interacao_necessidades",
        model="gemini-2.0-flash",
        instruction="""
        Você é um Agente de Interação e Necessidades para pessoas em situação de rua.
        Sua função é acolher o usuário, perguntar sobre sua necessidade imediata (ex: 'Preciso de abrigo', 'Estou com fome', 'Estou doente').
        Faça perguntas claras e diretas para entender a situação (ex: 'Você está sozinho(a)?', 'Qual sua localização atual?', 'Precisa de algo específico?').
        Ao final, retorne de forma estruturada a necessidade identificada do usuário e sua localização (se fornecida).
        """,
        description="Agente que interage com o usuário para entender suas necessidades.",
        # Não necessariamente precisa de ferramentas aqui, a menos que ele precise de busca simples para entender termos.
    )
    # A entrada para este agente será a mensagem direta do usuário (ex: "Estou com muito frio e não tenho onde dormir")
    # return call_agent(interacao, "Estou com muito frio e não tenho onde dormir.")
    return interacao

In [38]:
######################################
# --- Agente 3: Recomendador/Encaminhamento --- #
######################################
def agente_recomendacao_encaminhamento():
    recomendador = Agent(
        name="agente_recomendacao_encaminhamento",
        model="gemini-2.5-pro-preview-03-25", # Pode precisar de um modelo mais potente para raciocínio complexo
        instruction="""
        Você é um Agente de Recomendação e Encaminhamento.
        Recebe as necessidades de um usuário (tipo de necessidade, localização, etc.) e consulta o Agente de Mapeamento e Recursos (AMR)
        para encontrar as opções mais adequadas e disponíveis.
        Priorize recursos pela proximidade, relevância e condições específicas (ex: vagas em abrigos).
        Gere uma lista de 2-3 recomendações, incluindo nome do local, endereço completo, horário de funcionamento, como chegar (rota a pé/ônibus, se possível) e telefone de contato.
        Se não encontrar nada, indique que não há opções no momento.
        """,
        description="Agente que recomenda recursos com base nas necessidades do usuário e dados mapeados.",
        # Ele precisará de uma forma de "chamar" ou "consultar" o AMR.
        # Isso pode ser feito integrando a função do AMR aqui ou passando a informação.
        # Por simplicidade inicial, a informação do AMR pode ser passada como parte da entrada.
        tools=[google_search] # Pode ser útil para rotas ou detalhes adicionais
    )
    # A entrada para este agente seria algo como:
    # entrada = f"Necessidade do usuário: {necessidade_do_ain}\nRecursos disponíveis: {dados_do_amr}"
    # return call_agent(recomendador, entrada)
    return recomendador

In [39]:
##########################################
# --- Agente 4: Alertas e Clima --- #
##########################################
def agente_alertas_clima():
    alertas = Agent(
        name="agente_alertas_clima",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
        Você é um Agente de Alertas e Clima. Monitore proativamente as condições climáticas
        (frio/calor extremo, chuva intensa), alertas de saúde pública (campanhas de vacinação, surtos)
        e eventos emergenciais (distribuição de alimentos/cobertores, mutirões).
        Sua função é gerar alertas claros e concisos quando essas condições forem críticas.
        Indique a natureza do alerta e quais ações ou recursos podem ser relevantes (ex: 'Alerta de frio extremo! Busque abrigo!').
        """,
        description="Agente que monitora e emite alertas proativos sobre clima e eventos.",
        tools=[google_search] # Para buscar informações de clima/notícias de saúde pública
        # Poderia integrar APIs de clima diretamente aqui.
    )
    # A entrada para este agente poderia ser uma trigger baseada em tempo (a cada X horas)
    # return call_agent(alertas, f"Verifique alertas para hoje ({date.today().strftime('%d/%m/%Y')}).")
    return alertas

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Apoio a Pessoas em Situação de Rua com 4 Agentes 🚀")

# --- Instancia os Agentes ---
amr = agente_mapeamento_recursos()
ain = agente_interacao_necessidades()
are = agente_recomendacao_encaminhamento()
aac = agente_alertas_clima() # Este agente pode ser executado em paralelo ou periodicamente

# --- Interação com o Usuário ---
necessidade_usuario_raw = input("❓ Olá! Em que posso te ajudar hoje? (ex: 'Preciso de abrigo', 'Estou com fome', 'Estou doente') ")

if not necessidade_usuario_raw:
    print("Por favor, digite sua necessidade.")
else:
    print(f"Entendido! Você precisa de ajuda com: {necessidade_usuario_raw}")

    # --- 1. AIN compreende a necessidade ---
    print("\n--- 🗣️ Agente de Interação e Necessidades (AIN) analisando... ---")
    necessidade_identificada = call_agent(ain, necessidade_usuario_raw)
    display(to_markdown(necessidade_identificada))
    print("--------------------------------------------------------------")

    # --- 2. AMR busca/atualiza recursos (pode ser contínuo ou ativado por demanda) ---
    # Para o exemplo, vamos simular que ele busca recursos gerais ou específicos
    print("\n--- 🗺️ Agente de Mapeamento e Recursos (AMR) buscando recursos... ---")
    # Aqui, o AMR precisaria buscar os dados e retorná-los de forma estruturada.
    # Por exemplo, uma chamada que buscaria abrigos, restaurantes, etc.
    # O retorno idealmente seria um JSON ou lista de dicionários de recursos.
    # Para simplificar aqui, vamos pedir uma busca textual para demonstração.
    recursos_gerais = call_agent(amr, f"Liste os principais recursos de abrigo e alimentação na sua área de cobertura.")
    display(to_markdown(recursos_gerais))
    print("--------------------------------------------------------------")

    # --- 3. AAC verifica alertas (pode ser contínuo ou ativado por demanda) ---
    print("\n--- 🚨 Agente de Alertas e Clima (AAC) verificando alertas... ---")
    alerta_atual = call_agent(aac, f"Verifique alertas de clima ou eventos para {data_de_hoje} que possam afetar pessoas em situação de rua.")
    display(to_markdown(alerta_atual))
    print("--------------------------------------------------------------")

    # --- 4. ARE recomenda com base na necessidade, recursos e alertas ---
    print("\n--- 🎯 Agente de Recomendação e Encaminhamento (ARE) recomendando... ---")
    # O ARE precisaria receber tanto a necessidade do AIN quanto os recursos do AMR
    # e, opcionalmente, os alertas do AAC para ajustar a prioridade.
    # Esta é a parte mais complexa da orquestração. Você precisaria passar os dados
    # do `necessidade_identificada` e `recursos_gerais` para o ARE.
    entrada_para_are = f"Necessidade do usuário: {necessidade_identificada}\nRecursos disponíveis: {recursos_gerais}\nAlertas Atuais: {alerta_atual}"
    recomendacoes_finais = call_agent(are, entrada_para_are)
    display(to_markdown(recomendacoes_finais))
    print("--------------------------------------------------------------")

    print("\n✅ Processo de recomendação finalizado!")